# Data Preparation

In [1]:
!pwd

/home/hmp/Documents/My_Exp/word2vec


In [2]:
from tqdm import tqdm
# tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing
from sklearn import utils
import pandas as pd
pd.set_option('max.colwidth', 0)

In [3]:
df_wb = pd.read_csv("df_w2v.csv")
df_wb = df_wb.drop(columns=["Unnamed: 0"], axis=1)
df_wb

,burmese
0,ကောင်လေး တွေ ၊ အရမ်း ဝေးဝေး မှာ ရေ မ ကူးနဲ့'၊ ကျွန်တော် ပြော ခဲ့ တယ် ။
1,ကျုပ် တို့ ကို မျက်စိမြင် အောင် လုပ် ပေး ပါ ။
2,ကျွန်တော် ကြား ရ တာ စိတ် မ ကောင်း ပါ ဘူး ။
3,ကျွန်တော် ကြား ပါ တယ်
4,ကျွန်တော် ကြား ပါ တယ် ။
...,...
1177161,အဲဒီတော့ ရခိုင် ပါတီ ကို ။
1177162,အဲဒီလို လုပ် တဲ့ ပွဲ လို့ ထင် ပါ တယ် ။
1177163,ထိုင်း နဲ့ ဂျပန် နိုင်ငံ တွေ မှာ လည်း Virus ပိုးကူးစက် နေ သူ တွေ ရှိ တယ် ဆို တာ အတည်ပြု ထား ပြီး ဖြစ် ပါ တယ် ။
1177164,ဒီ အကြောင်း အေပီ သတင်း ဆောင်းပါး ကို အခြေခံ ပြီး ကို ဇော်ဝင်းလှိုင် က စုစည်း တင်ပြ ပေး ပါ မယ် ။


In [4]:
df_wb = df_wb.dropna(subset=["burmese"])
df_wb.isnull().sum()

burmese    0
dtype: int64

# Tag Documents

In [5]:
def tag_doc(df,label):
    result = []
    prefix = label
    for i, t in zip(df.index, df):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [6]:
tagged_docs = tag_doc(df_wb["burmese"],"doc")

In [7]:
tagged_docs[0:5]

[TaggedDocument(words=['ကောင်လေး', 'တွေ', '၊', 'အရမ်း', 'ဝေးဝေး', 'မှာ', 'ရေ', 'မ', "ကူးနဲ့'၊", 'ကျွန်တော်', 'ပြော', 'ခဲ့', 'တယ်', '။'], tags=['doc_0']),
 TaggedDocument(words=['ကျုပ်', 'တို့', 'ကို', 'မျက်စိမြင်', 'အောင်', 'လုပ်', 'ပေး', 'ပါ', '။'], tags=['doc_1']),
 TaggedDocument(words=['ကျွန်တော်', 'ကြား', 'ရ', 'တာ', 'စိတ်', 'မ', 'ကောင်း', 'ပါ', 'ဘူး', '။'], tags=['doc_2']),
 TaggedDocument(words=['ကျွန်တော်', 'ကြား', 'ပါ', 'တယ်'], tags=['doc_3']),
 TaggedDocument(words=['ကျွန်တော်', 'ကြား', 'ပါ', 'တယ်', '။'], tags=['doc_4'])]

In [8]:
words = tagged_docs[0].words
words

['ကောင်လေး',
 'တွေ',
 '၊',
 'အရမ်း',
 'ဝေးဝေး',
 'မှာ',
 'ရေ',
 'မ',
 "ကူးနဲ့'၊",
 'ကျွန်တော်',
 'ပြော',
 'ခဲ့',
 'တယ်',
 '။']

In [9]:
tags = words = tagged_docs[0].tags
tags

['doc_0']

## Train Word2Vec

In [10]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, vector_size=100, negative=5, window=5, workers=cores, alpha=0.065, min_alpha=0.001)
model_ug_cbow.build_vocab([x.words for x in tqdm(tagged_docs)])

100%|████████████████████████████| 1177165/1177165 [00:00<00:00, 5695741.70it/s]


In [11]:
%%time
for epoch in range(10):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(tagged_docs)]), total_examples=len(tagged_docs), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|████████████████████████████| 1177165/1177165 [00:00<00:00, 4111913.09it/s]


CPU times: user 4min 17s, sys: 1.08 s, total: 4min 18s
Wall time: 1min 12s


In [12]:
model_ug_cbow.wv.vectors.shape

(46537, 100)

In [13]:
model_ug_cbow.wv.most_similar("ပိုး")

[('ရောဂါပိုး', 0.7769202589988708),
 ('ကိုဗစ်ပိုး', 0.6159020662307739),
 ('ဗိုင်းရပ်စ်ပိုး', 0.6109343767166138),
 ('COVID-19ပိုး', 0.6010725498199463),
 ('လူနာ,', 0.559122622013092),
 ('ကာလသားရောဂါ', 0.5580489039421082),
 ('Positive', 0.5433117151260376),
 ('omicron', 0.5204574465751648),
 ('ဗစ်ပိုး', 0.5199772715568542),
 ('ရောဂါ', 0.5179102420806885)]

In [14]:
model_ug_cbow.wv.vector_size

100

In [15]:
words = list(model_ug_cbow.wv.index_to_key)
words[0]

'။'

In [16]:
words[954]

'တိုက်တွန်း'

In [17]:
model_ug_cbow.wv.most_similar('တိုက်တွန်း')

[('တောင်းဆို', 0.7532241344451904),
 ('ဖိတ်ခေါ်', 0.7122880220413208),
 ('မေတ္တာရပ်ခံ', 0.6929976344108582),
 ('အကြံပြု', 0.6583988666534424),
 ('အကြံပေး', 0.63477623462677),
 ('ပုံငွေ,', 0.6233166456222534),
 ('ပန်ကြား', 0.6017525792121887),
 ('သတိပေး', 0.5935220718383789),
 ('နှိုးဆော်', 0.5928245782852173),
 ('မေတ္တာရပ်', 0.5862002372741699)]

In [18]:
model_ug_cbow.save("word2vec.model")

## Test Word2Vec

In [19]:
w2v_model = Word2Vec.load("word2vec.model")

In [20]:
w2v_model.wv.most_similar('ဘောင်းဘီ')

[('အင်္ကျီ', 0.7127751111984253),
 ('စွပ်ကျယ်', 0.7051407098770142),
 ('လုံချည်', 0.6726532578468323),
 ('ပုဆိုး', 0.6664892435073853),
 ('လည်စီး', 0.6570024490356445),
 ('ရှပ်အင်္ကျီ', 0.6539551019668579),
 ('ဘရာစီယာ', 0.6517106294631958),
 ('အကျီ', 0.6481223702430725),
 ('လက်အိတ်', 0.6478937268257141),
 ('အဝတ်', 0.6324366331100464)]

In [21]:
w2v_model.wv["ဘောင်းဘီ"]

array([ 1.8429667 ,  0.14643562, -2.374249  ,  1.0118915 , -0.3351355 ,
       -0.59055865, -1.7698783 ,  0.84621626, -0.70421   , -2.0615008 ,
       -1.2275231 ,  0.31361023,  0.17070056, -0.9410883 , -0.8043827 ,
       -0.5222622 , -3.8169384 , -0.10094077,  0.40374812,  0.03341421,
        1.6709826 ,  1.0752032 ,  1.3575878 ,  0.990701  ,  1.8032225 ,
       -0.11063171,  0.03165381, -0.4741613 , -1.2987038 ,  0.9467315 ,
        0.40486717, -0.81629664,  1.2484583 , -1.2938097 , -0.41614515,
        1.2418358 , -0.47147048,  0.21659842, -1.6580931 , -1.3912206 ,
        1.0457327 ,  0.51607496, -2.4523098 , -2.7993329 , -1.2162846 ,
       -0.9557527 , -0.7400147 , -0.4302669 ,  1.0487834 , -1.6271943 ,
       -0.9770235 , -1.6658928 , -2.1336536 ,  0.02904725,  0.425     ,
       -0.07462347, -1.3748847 , -3.6821337 , -0.9104632 , -0.21138409,
        0.92463297, -0.26068452, -0.3186846 , -0.7852328 , -2.129254  ,
        2.5104585 , -1.2488754 ,  1.6020925 ,  0.8184464 ,  0.84

In [22]:
w2v_model.wv.most_similar('ဖျော်ရည်')

[('ကော်ဖီ', 0.7264184951782227),
 ('ရေနွေးကြမ်း', 0.6559270620346069),
 ('ပျားရည်', 0.6505799293518066),
 ('ဟင်းချို', 0.6462851166725159),
 ('ဒိန်ချဉ်', 0.6428185105323792),
 ('ဝိုင်', 0.6417125463485718),
 ('ကြိတ်ရည်', 0.6315672993659973),
 ('ရှာလကာရည်', 0.6190642714500427),
 ('polyphenol', 0.6149936318397522),
 ('နွားနို့', 0.6094685792922974)]

In [23]:
w2v_model.wv.most_similar('ဂီတ')

[('အနုပညာ', 0.6814081072807312),
 ('စာပေ', 0.6367932558059692),
 ('တေးသီချင်း', 0.5969644784927368),
 ('Rock', 0.5748250484466553),
 ('ဇာတ်သဘင်', 0.566178560256958),
 ('မော်ဒယ်လ်', 0.548032283782959),
 ('Rap', 0.5449516177177429),
 ('သီဆို', 0.5408194661140442),
 ('HipHop', 0.5341389775276184),
 ('ဂျက်ဇ်', 0.5306814312934875)]